In [15]:
import sys
from pathlib import Path
import importlib.util

here = Path.cwd().resolve()
repo_root = next((p for p in (here, *here.parents) if (p / "src" / "__init__.py").exists()), None)
if repo_root is None:
    raise RuntimeError(f"Can't find repo root from {here}")

In [16]:
# Library imports
import numpy as np 
import pandas as pd 
import torch 
import torch.optim as optim 
from pathlib import Path 
import json
import matplotlib.pyplot as plt


# Codebase imports
from src.models.lstm import LSTM 
from src.models.masked_lstm import MaskedLSTM
from src.models.trainer import Trainer
from src.models.masked_trainer import MaskedTrainer
from src.utils.loss_functions import RMSELoss
from src.utils.config_file_parser import config_file_parser
from src.utils.plot_predictions import plot_predictions
from src.utils.plot_losses import plot_loss_curves
from src.utils.datapipeline import DataPipeline
from src.utils.window_sizes.sliding_window_pipeline import SlidingWindowPipeline, SlidingWindowBatch
from src.interpretability.pfi import permutation_feature_importance
from src.interpretability.pfi_plots import plot_pfi_radar, plot_pfi_bar
from src.utils.metric_functions import rmse

# Sliding Window Analysis
Per sliding window size, run each feature ~100 times to see how the distribution of errors are

In [ ]:
# Change if needed
county_name = "Fresno"
rodent_flag = True
# Timestamp to track creation of run data
timestamp = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
config_path = Path("../../config/masked_lstm_config.ini")
if rodent_flag:
  data_path   = Path(f"../../data/merged_rodent_{county_name.lower()}_agg.csv")
  run_dir     = Path(f"../../data/runs/{county_name.lower()}_Rat_{timestamp}")
else:
    data_path  = Path(f"../../data/{county_name.lower()}_Aggregate.csv")
    run_dir    = Path(f"../../data/runs/{county_name.lower()}_noRat_{timestamp}")

# run_dir.mkdir(parents=True, exist_ok=True)

# ------------- DO NOT TOUCH BELOW HERE --------------

# load config data
lstm_params, pipeline_params = config_file_parser(config_path=config_path)
hidden_size                  = int(lstm_params["hidden_size"])
num_layers                   = int(lstm_params["num_layers"])
dropout                      = float(lstm_params["dropout"])
learning_rate                = float(lstm_params["learning_rate"])
epochs                       = int(lstm_params["epochs"])
weight_decay                 = float(lstm_params["weight_decay"])
train_frac                   = float(lstm_params["train_frac"])
test_frac                    = 1 - train_frac

# import and clean the dataframe (remove date)
df = pd.read_csv(data_path)
for col in ["YEAR_MONTH", "Year-Month", "DATE"]:
    if col in df.columns:
      df = df.drop(columns = [col])

# Isolate the features I want
feature_columns = [col for col in df.columns if col != "VFRate"]

# Create the feature and target vectors
X = df[feature_columns].values
y = df["VFRate"].values 

########################################
#       Sliding Window Pipeline        #
########################################
# Create the datapipeline for sliding window calculations
pipeline = SlidingWindowPipeline(X, y, test_frac=test_frac)

# create a list of sliding window sizes from 1 to 12
sliding_window_sizes = [x for x in range(1,13)]
criterion = RMSELoss()

results = [] 
num_repeats = 100

for feat_idx, feat_name in enumerate(feature_columns):
  for win_size in sliding_window_sizes:
    batch : SlidingWindowBatch = pipeline.scale(*pipeline.create_single_feature_sequences(feat_idx, win_size))
    rmse_sw = []
    for run_idx in range(num_repeats):
      model = LSTM(
        input_size = 1,
        hidden_size= hidden_size, 
        num_layers= num_layers,
        dropout= dropout
      )
      
      optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
      trainer = Trainer(model=model, criterion=criterion, optimizer=optimizer, scaler_y = batch.scaler_y)
      
      trainer.train(X_train = batch.X_train, y_train = batch.y_train, epochs=epochs)
      preds, true = trainer.evaluate(batch.X_test, batch.y_test)
      
      rmse_sw.append(np.sqrt(np.mean((preds - true)**2)))
      
    results.append({
      "feature": feat_name, 
      "window_size": win_size,
      "rmse": rmse_sw
    })
    print(f"{feat_name:20s} | w={win_size:2d} | mean RMSE = {np.mean(rmse_sw):.4f}")

results_df = pd.DataFrame(results)
print(results_df)
# best_vals = results_df.loc[results_df.groupby("feature")["rmse"].idxmin()]

FIRE_Acres_Burned    | w= 1 | mean RMSE = 2.6641
FIRE_Acres_Burned    | w= 2 | mean RMSE = 2.5770
FIRE_Acres_Burned    | w= 3 | mean RMSE = 2.2309
FIRE_Acres_Burned    | w= 4 | mean RMSE = 2.1757
FIRE_Acres_Burned    | w= 5 | mean RMSE = 2.3753
FIRE_Acres_Burned    | w= 6 | mean RMSE = 2.4369
FIRE_Acres_Burned    | w= 7 | mean RMSE = 1.7642
FIRE_Acres_Burned    | w= 8 | mean RMSE = 1.6235
FIRE_Acres_Burned    | w= 9 | mean RMSE = 1.4129
FIRE_Acres_Burned    | w=10 | mean RMSE = 1.4600
FIRE_Acres_Burned    | w=11 | mean RMSE = 1.3181
FIRE_Acres_Burned    | w=12 | mean RMSE = 1.4414
PRECIP               | w= 1 | mean RMSE = 2.7587
PRECIP               | w= 2 | mean RMSE = 2.8007
PRECIP               | w= 3 | mean RMSE = 2.8081
PRECIP               | w= 4 | mean RMSE = 2.7655
PRECIP               | w= 5 | mean RMSE = 2.8047
PRECIP               | w= 6 | mean RMSE = 2.5625
PRECIP               | w= 7 | mean RMSE = 2.9811
PRECIP               | w= 8 | mean RMSE = 2.6868
PRECIP              